In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf

In [ ]:
#The ranks and x_cols were outputted by the Feature Selection scripts, the x_cols have had low variance features and highly correlated features already removed
response = 'label'
ranks_no = 20
%store -r Xy
%store -r df_ranks
ranks = df_ranks

In [ ]:
Xy[response].value_counts()
Xy.reset_index(drop = True, inplace = True)

In [ ]:
top_n_ranked_feats = ranks['feature_name'][0:ranks_no].tolist()
print(top_n_ranked_feats)

Subset data into that for training and evaluation and then that for predicting and plotting to graphically evaluate model performancem

In [ ]:
data_e_for_predicting = Xy.iloc[(Xy.shape[0]-9044):(Xy.shape[0])]
train_eval_data = Xy.iloc[0:(Xy.shape[0]-9044)]

data_e_for_predicting.reset_index(drop = True, inplace=True)

print(train_eval_data.shape)
print(data_e_for_predicting.shape)

label = train_eval_data[response]
feat = train_eval_data.drop(columns = response)

label_data_e = data_e_for_predicting[response]
feat_data_e = data_e_for_predicting.drop(columns = response)


Create tensorflow objects for the features, thes contain indexes to the feature columns within the dataframe

In [ ]:
tf_column_objects = []
for col in feat.columns.values:
    tf_column_objects.append(tf.feature_column.numeric_column(col))

Split up data fro training and testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feat, label, test_size=0.3)

In [ ]:
input_function = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train, batch_size=10,num_epochs=5, shuffle=True)



classifier = tf.estimator.DNNClassifier(hidden_units=[10, 200, 200, 10], n_classes=2, feature_columns = tf_column_objects)



In [ ]:


classifier.train(input_fn=input_function, steps=50)



In [ ]:
make predictions for test data and for data that will be used to assess model performance graphically

In [ ]:
pred_fn_eval = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)

pred_fn_data_e = tf.estimator.inputs.pandas_input_fn(x = feat_data_e, batch_size = len(feat_data_e), shuffle = False)

In [ ]:
predictions = list(classifier.predict(input_fn=pred_fn_eval))

predictions_data_e = list(classifier.predict(input_fn=pred_fn_data_e))

In [ ]:

Extract predictions of classes

In [ ]:


final_predictions =[]
for i in predictions:
    final_predictions.append(i['class_ids'][0])
    
final_predictions_data_e =[]
for i in predictions_data_e:
    final_predictions_data_e.append(i['class_ids'][0])

